In [46]:
import pandas as pd
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import re
from nltk.stem import PorterStemmer
from nltk import word_tokenize
from nltk.corpus import stopwords 
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
%matplotlib inline
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt


wine = pd.read_csv('wine_stem.csv')
wine.drop(['Unnamed: 0', 'region2'], axis = 1, inplace = True)

duplicated_records = wine[wine.duplicated(['countri', 'descript', 'design', 'point', 'price', 'provinc', 'region1', 'varieti', 'wineri'])]
duplicated_records.shape
new_wine = wine.drop_duplicates()
new_wine.shape

new_wine = new_wine.dropna(subset=['price','countri', 'provinc'])
new_wine.shape

wine_df=new_wine.loc[new_wine.countri=='us',['descript','provinc','design','point','price','region1','varieti','wineri']]

from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import KFold
from sklearn.model_selection import train_test_split

wine_df=new_wine.loc[new_wine.countri=='us',['descript','provinc','design','point','price','region1','varieti','wineri']]

wine_train = wine_df

design_lab=LabelEncoder().fit(np.unique(new_wine.design.values))
prov_lab=LabelEncoder().fit(np.unique(new_wine.provinc.values))
regn_lab=LabelEncoder().fit(np.unique(new_wine.region1.values))
var_lab=LabelEncoder().fit(np.unique(new_wine.varieti.values))
wnry_lab=LabelEncoder().fit(np.unique(new_wine.wineri.values))
points_lab=LabelEncoder().fit(np.unique(new_wine.point.values))
wine_train['prov_enc']=prov_lab.transform(wine_train.provinc)
wine_train['design_enc']=design_lab.transform(wine_train.design)
wine_train['regn_enc']=regn_lab.transform(wine_train.region1)
wine_train['var_enc']=var_lab.transform(wine_train.varieti)
wine_train['wnry_enc']=wnry_lab.transform(wine_train.wineri)
wine_train['points_enc']=points_lab.transform(wine_train.point)
wine_train=wine_train.drop(['provinc','design','point','region1','varieti','wineri'],axis=1)

bin_map={
    2300:'extreme expensive',
    2013:'extreme expensive',
    1900:'extreme expensive',
    1400:'extreme expensive',
    1400:'extreme expensive',
    1400:'extreme expensive',
    1300:'extreme expensive',
    1200:'extreme expensive',
    1100:'extreme expensive',
    1100:'extreme expensive',
    1000:'extreme expensive',
    980:'very expensive',
900:'very expensive',
850:'very expensive',
850:'very expensive',
850:'very expensive',
848:'very expensive',
800:'very expensive',
800:'very expensive',
800:'very expensive',
800:'very expensive',
775:'very expensive',
775:'very expensive',
770:'very expensive',
770:'very expensive',
764:'very expensive',
757:'very expensive',
757:'very expensive',
757:'very expensive',
740:'very expensive',
698:'very expensive',
685:'very expensive',
670:'very expensive',
670:'very expensive',
660:'very expensive',
660:'very expensive',
660:'very expensive',
660:'very expensive',
639:'very expensive',
639:'very expensive',
625:'very expensive',
625:'very expensive',
625:'very expensive',
625:'very expensive',
612:'very expensive',
600:'very expensive',
600:'very expensive',
600:'very expensive',
600:'very expensive',
600:'very expensive',
600:'very expensive',
599:'very expensive',
596:'very expensive',
596:'very expensive',
588:'very expensive',
580:'very expensive',
574:'very expensive',
550:'very expensive',
550:'very expensive',
550:'very expensive',
550:'very expensive',
545:'very expensive',
544:'very expensive',
535:'very expensive',
535:'very expensive',
535:'very expensive',
530:'very expensive',
525:'very expensive',
520:'very expensive',
510:'very expensive',
505:'very expensive',
505:'very expensive',
505:'very expensive',
100:'expensive',
101:'expensive',
102:'expensive',
103:'expensive',
104:'expensive',
105:'expensive',
106:'expensive',
107:'expensive',
108:'expensive',
109:'expensive',
110:'expensive',
111:'expensive',
112:'expensive',
113:'expensive',
114:'expensive',
115:'expensive',
116:'expensive',
117:'expensive',
118:'expensive',
119:'expensive',
120:'expensive',
121:'expensive',
122:'expensive',
123:'expensive',
124:'expensive',
125:'expensive',
126:'expensive',
127:'expensive',
128:'expensive',
129:'expensive',
130:'expensive',
131:'expensive',
132:'expensive',
133:'expensive',
134:'expensive',
135:'expensive',
136:'expensive',
137:'expensive',
138:'expensive',
139:'expensive',
140:'expensive',
141:'expensive',
142:'expensive',
143:'expensive',
144:'expensive',
145:'expensive',
146:'expensive',
147:'expensive',
148:'expensive',
149:'expensive',
150:'expensive',
151:'expensive',
152:'expensive',
153:'expensive',
154:'expensive',
155:'expensive',
156:'expensive',
157:'expensive',
158:'expensive',
159:'expensive',
160:'expensive',
161:'expensive',
162:'expensive',
163:'expensive',
164:'expensive',
165:'expensive',
166:'expensive',
167:'expensive',
168:'expensive',
169:'expensive',
170:'expensive',
171:'expensive',
172:'expensive',
173:'expensive',
174:'expensive',
175:'expensive',
176:'expensive',
177:'expensive',
178:'expensive',
179:'expensive',
180:'expensive',
181:'expensive',
182:'expensive',
183:'expensive',
184:'expensive',
185:'expensive',
186:'expensive',
187:'expensive',
188:'expensive',
189:'expensive',
190:'expensive',
191:'expensive',
192:'expensive',
193:'expensive',
194:'expensive',
195:'expensive',
196:'expensive',
197:'expensive',
198:'expensive',
199:'expensive',
200:'expensive',
201:'expensive',
202:'expensive',
203:'expensive',
204:'expensive',
205:'expensive',
206:'expensive',
207:'expensive',
208:'expensive',
209:'expensive',
210:'expensive',
211:'expensive',
212:'expensive',
213:'expensive',
214:'expensive',
215:'expensive',
216:'expensive',
217:'expensive',
218:'expensive',
219:'expensive',
220:'expensive',
221:'expensive',
222:'expensive',
223:'expensive',
224:'expensive',
225:'expensive',
226:'expensive',
227:'expensive',
228:'expensive',
229:'expensive',
230:'expensive',
231:'expensive',
232:'expensive',
233:'expensive',
234:'expensive',
235:'expensive',
236:'expensive',
237:'expensive',
238:'expensive',
239:'expensive',
240:'expensive',
241:'expensive',
242:'expensive',
243:'expensive',
244:'expensive',
245:'expensive',
246:'expensive',
247:'expensive',
248:'expensive',
249:'expensive',
250:'expensive',
251:'expensive',
252:'expensive',
253:'expensive',
254:'expensive',
255:'expensive',
256:'expensive',
257:'expensive',
258:'expensive',
259:'expensive',
260:'expensive',
261:'expensive',
262:'expensive',
263:'expensive',
264:'expensive',
265:'expensive',
266:'expensive',
267:'expensive',
268:'expensive',
269:'expensive',
270:'expensive',
271:'expensive',
272:'expensive',
273:'expensive',
274:'expensive',
275:'expensive',
276:'expensive',
277:'expensive',
278:'expensive',
279:'expensive',
280:'expensive',
281:'expensive',
282:'expensive',
283:'expensive',
284:'expensive',
285:'expensive',
286:'expensive',
287:'expensive',
288:'expensive',
289:'expensive',
290:'expensive',
291:'expensive',
292:'expensive',
293:'expensive',
294:'expensive',
295:'expensive',
296:'expensive',
297:'expensive',
298:'expensive',
299:'expensive',
300:'expensive',
301:'expensive',
302:'expensive',
303:'expensive',
304:'expensive',
305:'expensive',
306:'expensive',
307:'expensive',
308:'expensive',
309:'expensive',
310:'expensive',
311:'expensive',
312:'expensive',
313:'expensive',
314:'expensive',
315:'expensive',
316:'expensive',
317:'expensive',
318:'expensive',
319:'expensive',
320:'expensive',
321:'expensive',
322:'expensive',
323:'expensive',
324:'expensive',
325:'expensive',
326:'expensive',
327:'expensive',
328:'expensive',
329:'expensive',
330:'expensive',
331:'expensive',
332:'expensive',
333:'expensive',
334:'expensive',
335:'expensive',
336:'expensive',
337:'expensive',
338:'expensive',
339:'expensive',
340:'expensive',
341:'expensive',
342:'expensive',
343:'expensive',
344:'expensive',
345:'expensive',
346:'expensive',
347:'expensive',
348:'expensive',
349:'expensive',
350:'expensive',
351:'expensive',
352:'expensive',
353:'expensive',
354:'expensive',
355:'expensive',
356:'expensive',
357:'expensive',
358:'expensive',
359:'expensive',
360:'expensive',
361:'expensive',
362:'expensive',
363:'expensive',
364:'expensive',
365:'expensive',
366:'expensive',
367:'expensive',
368:'expensive',
369:'expensive',
370:'expensive',
371:'expensive',
372:'expensive',
373:'expensive',
374:'expensive',
375:'expensive',
376:'expensive',
377:'expensive',
378:'expensive',
379:'expensive',
380:'expensive',
381:'expensive',
382:'expensive',
383:'expensive',
384:'expensive',
385:'expensive',
386:'expensive',
387:'expensive',
388:'expensive',
389:'expensive',
390:'expensive',
391:'expensive',
392:'expensive',
393:'expensive',
394:'expensive',
395:'expensive',
396:'expensive',
397:'expensive',
398:'expensive',
399:'expensive',
400:'expensive',
401:'expensive',
402:'expensive',
403:'expensive',
404:'expensive',
405:'expensive',
406:'expensive',
407:'expensive',
408:'expensive',
409:'expensive',
410:'expensive',
411:'expensive',
412:'expensive',
413:'expensive',
414:'expensive',
415:'expensive',
416:'expensive',
417:'expensive',
418:'expensive',
419:'expensive',
420:'expensive',
421:'expensive',
422:'expensive',
423:'expensive',
424:'expensive',
425:'expensive',
426:'expensive',
427:'expensive',
428:'expensive',
429:'expensive',
430:'expensive',
431:'expensive',
432:'expensive',
433:'expensive',
434:'expensive',
435:'expensive',
436:'expensive',
437:'expensive',
438:'expensive',
439:'expensive',
440:'expensive',
441:'expensive',
442:'expensive',
443:'expensive',
444:'expensive',
445:'expensive',
446:'expensive',
447:'expensive',
448:'expensive',
449:'expensive',
450:'expensive',
451:'expensive',
452:'expensive',
453:'expensive',
454:'expensive',
455:'expensive',
456:'expensive',
457:'expensive',
458:'expensive',
459:'expensive',
460:'expensive',
461:'expensive',
462:'expensive',
463:'expensive',
464:'expensive',
465:'expensive',
466:'expensive',
467:'expensive',
468:'expensive',
469:'expensive',
470:'expensive',
471:'expensive',
472:'expensive',
473:'expensive',
474:'expensive',
475:'expensive',
476:'expensive',
477:'expensive',
478:'expensive',
479:'expensive',
480:'expensive',
481:'expensive',
482:'expensive',
483:'expensive',
484:'expensive',
485:'expensive',
486:'expensive',
487:'expensive',
488:'expensive',
489:'expensive',
490:'expensive',
491:'expensive',
492:'expensive',
493:'expensive',
494:'expensive',
495:'expensive',
496:'expensive',
497:'expensive',
498:'expensive',
499:'expensive',
    500:'expensive',
    50:'normal',
51:'normal',
52:'normal',
53:'normal',
54:'normal',
55:'normal',
56:'normal',
57:'normal',
58:'normal',
59:'normal',
60:'normal',
61:'normal',
62:'normal',
63:'normal',
64:'normal',
65:'normal',
66:'normal',
67:'normal',
68:'normal',
69:'normal',
70:'normal',
71:'normal',
72:'normal',
73:'normal',
74:'normal',
75:'normal',
76:'normal',
77:'normal',
78:'normal',
79:'normal',
80:'normal',
81:'normal',
82:'normal',
83:'normal',
84:'normal',
85:'normal',
86:'normal',
87:'normal',
88:'normal',
89:'normal',
90:'normal',
91:'normal',
92:'normal',
93:'normal',
94:'normal',
95:'normal',
96:'normal',
97:'normal',
98:'normal',
99:'normal',
100:'normal',
    1:'cheap',
2:'cheap',
3:'cheap',
4:'cheap',
5:'cheap',
6:'cheap',
7:'cheap',
8:'cheap',
9:'cheap',
10:'cheap',
11:'cheap',
12:'cheap',
13:'cheap',
14:'cheap',
15:'cheap',
16:'cheap',
17:'cheap',
18:'cheap',
19:'cheap',
20:'cheap',
21:'cheap',
22:'cheap',
23:'cheap',
24:'cheap',
25:'cheap',
26:'cheap',
27:'cheap',
28:'cheap',
29:'cheap',
30:'cheap',
31:'cheap',
32:'cheap',
33:'cheap',
34:'cheap',
35:'cheap',
36:'cheap',
37:'cheap',
38:'cheap',
39:'cheap',
40:'cheap',
41:'cheap',
42:'cheap',
43:'cheap',
44:'cheap',
45:'cheap',
46:'cheap',
47:'cheap',
48:'cheap',
49:'cheap'
    
    
}
wine_train['price_bins']=wine_train.price.map(bin_map)
wine_train.price_bins.unique() # Ensure no records are un-binned
wine_train=wine_train.drop('price',axis=1)
price_lab=LabelEncoder().fit(np.unique(wine_train.price_bins.values))
wine_train['price_enc']=price_lab.transform(wine_train.price_bins)
wine_train=wine_train.drop(['price_bins'],axis=1)

In [47]:
wine_train

,descript,prov_enc,design_enc,regn_enc,var_enc,wnry_enc,points_enc,price_enc
0,tremend 100 variet wine hail oakvill age three...,51,15621,700,69,6621,16,1
2,mac watson honor memori wine made mother treme...,51,23130,498,456,8252,16,3
3,spent 20 month 30 new french oak incorpor frui...,279,20190,1167,392,10154,16,3
8,renam vineyard former bottl delancellotti youl...,279,22566,190,392,988,15,3
9,produc sourc two block vineyard wineon high el...,51,9974,950,392,1136,15,3
11,18yearold vine suppl wellbalanc effort blend f...,279,8829,826,392,9719,15,0
12,standout even terrif lineup 2015 releas patric...,279,26539,375,392,9719,15,0
14,sophist mix miner acid tart fruit seduct effor...,279,10393,375,392,4936,15,1
15,first made 2006 succul luscious chardonnay min...,279,22554,1167,109,988,15,3
16,blockbust powerhous wine suggest blueberri pie...,51,19779,361,69,6491,15,1


In [70]:
ar = wine_train['prov_enc'].unique()
ar.sort()
regn_decode = ""
print ar
for num in ar:
    predictions_test = prov_lab.inverse_transform(num)
    regn_decode += (" " + str(num) + " : "+ str(predictions_test))
   
regn_decode

[ 12  19  51  84  86 154 158 172 217 222 226 259 260 261 263 266 274 279
 291 378 405 421 432 433]


' 12 : america 19 : arizona 51 : california 84 : colorado 86 : connecticut 154 : idaho 158 : iowa 172 : kentucki 217 : massachusett 222 : michigan 226 : missouri 259 : nevada 260 : new jersey 261 : new mexico 263 : new york 266 : north carolina 274 : ohio 279 : oregon 291 : pennsylvania 378 : texa 405 : vermont 421 : virginia 432 : washington 433 : washingtonoregon'

In [48]:
def normalize(text):
    letters = re.sub("[^a-zA-Z]", " ", text)
    words = letters.lower()
    return words

In [49]:
processed_descriptions = []
for description in wine_train['descript']:
    norm = normalize(description)
    processed_descriptions.append(norm)

tfidf_vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(1,2), min_df=3)
tfidf_matrix = tfidf_vectorizer.fit_transform(processed_descriptions)
feature_names = tfidf_vectorizer.get_feature_names()
doc = 10
feature_index = tfidf_matrix[:,:].nonzero()[1]
tfidf_scores = zip(feature_index, sorted([tfidf_matrix[doc, x] for x in feature_index]))

In [57]:
# function were taken from here: https://buhrmann.github.io/tfidf-analysis.html
def top_tfidf_feats(row, features, n_top=10):
    ''' Get top n tfidf values in row and return them with their corresponding feature names.'''
    topn_ids = np.argsort(row)[::-1][:10]
    top_feats = [(features[i], row[i]) for i in topn_ids]
    df = pd.DataFrame(top_feats)
    df.columns = ['feature', 'tfidf']
    return top_feats

def top_feats_in_doc(Xtr, features, row_id, n_top=10):
    ''' Top tfidf features in specific document (matrix row) '''
    row = np.squeeze(Xtr[row_id].toarray())
    return top_tfidf_feats(row, features,n_top=10)

In [92]:
import time
start_time = time.time()

# limit dataset to the more popular wine varieties
# df_variety = new_wine.groupby('varieti').filter(lambda x: len(x) > 5000)

word_scores = []
for i, j in enumerate(wine_train['price_enc']):
    for item in top_feats_in_doc(tfidf_matrix, feature_names, i):
        word_scores.append(item + (j,))

df = pd.DataFrame(word_scores)
df.columns = ['feature', 'tfidf', 'price_enc']
# df = df.sort_values('tfidf', ascending=False)
elapsed_time = time.time() - start_time
print elapsed_time


63.4866621494


In [93]:
df

,feature,tfidf,price_enc
0,compel hint,0.209871,1
1,year oak,0.209871,1
2,frame eleg,0.205288,1
3,redcherri fruit,0.205288,1
4,balanc reward,0.205288,1
5,year ahead,0.201544,1
6,tannin subtl,0.195636,1
7,fruit compel,0.195636,1
8,oak juici,0.191053,1
9,finish year,0.187308,1


In [97]:
new_df = df[df.price_enc==4]
new_df = new_df.append(df[df.price_enc == 1][:10])
df

,feature,tfidf,price_enc
0,compel hint,0.209871,1
1,year oak,0.209871,1
2,frame eleg,0.205288,1
3,redcherri fruit,0.205288,1
4,balanc reward,0.205288,1
5,year ahead,0.201544,1
6,tannin subtl,0.195636,1
7,fruit compel,0.195636,1
8,oak juici,0.191053,1
9,finish year,0.187308,1


In [98]:
feature_lab=LabelEncoder().fit(np.unique(df.feature.values))
df['feature_enc']=feature_lab.transform(df.feature)
df.drop(['feature'], axis=1)

,tfidf,price_enc,feature_enc
0,0.209871,1,11231
1,0.209871,1,57204
2,0.205288,1,20572
3,0.205288,1,40634
4,0.205288,1,3341
5,0.201544,1,57116
6,0.195636,1,50275
7,0.195636,1,21107
8,0.191053,1,34373
9,0.187308,1,18750


In [103]:
df = df[:1000][:]

In [104]:
df

,feature,tfidf,price_enc,feature_enc
0,compel hint,0.209871,1,11231
1,year oak,0.209871,1,57204
2,frame eleg,0.205288,1,20572
3,redcherri fruit,0.205288,1,40634
4,balanc reward,0.205288,1,3341
5,year ahead,0.201544,1,57116
6,tannin subtl,0.195636,1,50275
7,fruit compel,0.195636,1,21107
8,oak juici,0.191053,1,34373
9,finish year,0.187308,1,18750


In [105]:
from sklearn.model_selection import KFold
from sklearn import tree
start_time = time.time()
X,y = df[['feature_enc']], df['price_enc']
from sklearn.tree import DecisionTreeClassifier
for depth in range(1,10):
    tree_classifier = DecisionTreeClassifier(
        max_depth=3, random_state=0)
    if tree_classifier.fit(X,y).tree_.max_depth < depth:
        break

kf = KFold(n_splits=10)
kf.get_n_splits(X)
scores = cross_val_score(tree_classifier, X, y, cv=10)
tree.export_graphviz(tree_classifier, out_file='tfidfdecisiontree.dot')
print time.time() - start_time
print scores.sum()/10

0.0401120185852
0.667


In [106]:
headers = list(df)
with open("tfidfdecisiontree.dot", "w") as output_file:
    tree.export_graphviz(tree_classifier, feature_names=headers,  class_names = ['cheap','normal','expensive','very expensive','extreme expensive'],
                         out_file=output_file)

In [82]:
ar = df['feature_enc'].unique()
ar.sort()
regn_decode = ""
print ar
for num in ar:
    predictions_test = feature_lab.inverse_transform(num)
    regn_decode += (" " + str(num) + " : "+ str(predictions_test))
   
regn_decode

[    0     1     2 ..., 34835 34836 34837]


' 0 : abbey 1 : abbott 2 : abeja 3 : abil 4 : abil age 5 : abil ripen 6 : abl 7 : abl age 8 : abl handl 9 : abound 10 : abrupt 11 : absenc 12 : absenc fruit 13 : absenc structur 14 : absolut 15 : absolut beauti 16 : absolut decant 17 : absolut delici 18 : absolut dri 19 : absolut firstrat 20 : absolut gorgeous 21 : absolut love 22 : absolut stun 23 : absorb 24 : absorb new 25 : abund 26 : abund black 27 : abund cherri 28 : abund coffe 29 : abund fruit 30 : abund red 31 : abund vanilla 32 : abv 33 : accent add 34 : accent anis 35 : accent balanc 36 : accent barrel 37 : accent black 38 : accent blackberri 39 : accent bodi 40 : accent bright 41 : accent butter 42 : accent caramel 43 : accent cedar 44 : accent cinnamon 45 : accent citrus 46 : accent cocoa 47 : accent coffe 48 : accent cola 49 : accent cranberri 50 : accent crisp 51 : accent dark 52 : accent deep 53 : accent dri 54 : accent earthi 55 : accent fine 56 : accent finish 57 : accent firm 58 : accent floral 59 : accent fresh 60 :